## Read in data

In [1]:
import pandas as pd
import pickle
from datetime import datetime
from time import mktime
from nltk.metrics import edit_distance
from siuba import *

%load_ext blackcellmagic
#!jupyter labextension install @lckr/jupyterlab_variableinspector

In [2]:
lib1 = pickle.load(open('../data/2016-07-15/itl.p', 'rb'))
lib2 = pickle.load(open('../data/2019-07-31/itl.p', 'rb'))

raw_data1 = [s[1].ToDict() for s in lib1.songs.items()]
raw_data2 = [s[1].ToDict() for s in lib2.songs.items()]

In [3]:
convert_date = (_
    .dropna()
    .apply(lambda x: datetime.fromtimestamp(mktime(x)))
    .astype("datetime64[ns]")
)

clean_all = (
    mutate(
        date_added = convert_date(_.date_added),
        date_modified = convert_date(_.date_modified),
        skip_date = convert_date(_.skip_date)
    ) >>
    select(
        _.contains("date"),
        _.artist, _.name, _.album, _.genre, _.kind,
        _.persistent_id, _.play_count,
        _.rating, _.skip_count, _.track_id,
        _.track_number, _.year
    ) >>
    filter(~_.date_added.isna()) # Shared playlist songs shouldn't be included
)


data1 = (
    pd.DataFrame(raw_data1) >>
    clean_all
)

data2 = (
    pd.DataFrame(raw_data2) >>
    clean_all
)

In [4]:
(
    data2 >>
    count(added_year = _.date_added.dt.year)
)

,added_year,n
0,2006,3
1,2007,2
2,2008,3
3,2009,3
4,2010,6
5,2011,133
6,2012,171
7,2013,114
8,2014,48
9,2015,48


In [105]:
pickle.dump(data1, open("../data/data1.p", "wb"))
pickle.dump(data2, open("../data/data2.p", "wb"))

## Match songs from 2016 <=> 2019

There should be three dataframes that will need to be concatenated:
1. `joined_on_pid`, the songs that were not changed (they had the same persistent_id in 2016 as 2019);
2. `joined`, the songs that were matched (exactly) using artist and name (excluding `joined_on_pid`);
3. And `manually_joined`, the songs from `data1` that are not included in the above DFs that are manually matched to new versions.

The goal is ensure all the songs pre-Apple music have accurate metadata. Songs added after the switch to AM are almost exclusively AM songs.

In [4]:
# Use persistent_id to match songs, then remove those from future attempts to join
# Note: track id is not enough to use as a join key
joined_on_pid = (
    data1 >>
    inner_join(_, data2, on = "persistent_id") >>
    pipe(lambda d: d[sorted(d.columns)])
)

In [5]:
joined_raw = (
    data1
    # >> anti_join(joined_on_pid, on = 'persistent_id')
    >> left_join(
        _,
        joined_on_pid >> transmute(_.persistent_id, in_pid_table=True),
        on="persistent_id",
    )
    >> filter(_.in_pid_table.isna())
    >> inner_join(_, data2, on=["name", "artist"])
    >> pipe(lambda d: d[sorted(d.columns)])
)

In [6]:
# Manually correct problem children from the join
pd.options.display.max_columns = None
from siuba.dply.vector import n
# n(_) is the siuba equivalent of shape[0]

# We will be looping over persistent_id_y.
# Manually confirm in a CSV that all of the songs matched to a single persistent_id_y should in fact be matched.
(
    joined_raw 
    >> group_by(_.persistent_id_y)
    >> mutate(n = _.shape[0]) 
    >> ungroup()
    >> arrange(-_.n, _.artist, _.name)
#     >> filter(_.n > 1)
).to_csv('../data/manual_cleaning/joined_raw.csv')

In [7]:
# Read in cleaned CSV
joined_clean = (
    pd.read_csv('../data/manual_cleaning/joined_clean.csv') 
    >> filter(_.remove != 1) 
    >> select(-_.remove)
)

In [8]:
# Get everything from data1 that is not in the above two DataFrames.
needs_to_be_joined = data1 >> filter(
    ~_.persistent_id.isin(joined_on_pid.persistent_id),
    ~_.persistent_id.isin(joined_clean.persistent_id_x),
)

In [114]:
pickle.dump(
    needs_to_be_joined
    >> rename(persistent_id_x = "persistent_id")
    >> mutate(persistent_id_y = None), 
    open("../data/manual_cleaning/needs_to_by_joined.p", "wb")
)

In [9]:
# Take edit_distance out for a spin
def calculate_distance(s): 
    return (_
        .dropna()
        .apply(lambda x: edit_distance(x, s))
    )
artist_distance = calculate_distance('Deux')

test = (
    joined_on_pid
    >> mutate(distance=artist_distance(_.artist_x))
#     >> mutate(artist_distance_ntile=pd.qcut(_.distance, 10, labels=list(range(10))))
)

In [11]:
# Read in cleaned CSV
# ERRORS: sixteen, what a wonderful world, goyim friends, concert for george (got auto-matched)
# Add back music i've made (roundup, stockade, mashups)
# Add Longest Road (original) to the right playlists 
# added post hoc: john lennon through the night, Sinatra Witchcraft, Rihanna Disturbia, Slim Thug How we Do It, Drake Mary's Song, Kendrick County Building Blues, 

In [10]:
# Prototype the final thing - combining all sources into a master list of changes
painstakingly_matched = pickle.load(
    open("../data/manual_cleaning/needs_to_by_joined_wip.p", "rb")
) >> filter(~_.persistent_id_y.isna())

select_cols = (
    select(
        _.contains("persistent_id"),
        _.contains("artist"),
        _.contains("album"),
        _.contains("name"),
        _.contains("date_added"),
        _.contains("play_count"),
        _.contains("skip_count"),
        _.contains("genre"),
    ) 
    >> pipe(lambda d: d[sorted(d.columns)])
)

master_changes_raw = pd.concat(
    [
        (
            painstakingly_matched
            >> inner_join(
                _,
                data2 >> rename(persistent_id_y="persistent_id"),
                on="persistent_id_y",
            )
            >> select_cols
            >> mutate(src="painstakingly_matched")
        ),
        (
            joined_on_pid
            >> mutate(persistent_id_x=_.persistent_id, persistent_id_y=_.persistent_id)
            >> select(-_.persistent_id)
            >> select_cols
            >> mutate(src="joined_on_pid")
        ),
        # Because it is joined on artist and name, they will not be appended with _x and _y
        (
            joined_clean
            >> mutate(
                artist_x=_.artist, artist_y=_.artist, name_x=_.name, name_y=_.name
            )
            >> select(-_.name, -_.artist, -_["Unnamed: 30"])
            >> select_cols
            >> mutate(src="joined_clean")
        ),
    ],
    ignore_index=True
)

In [11]:
# Health checks
master_changes_raw >> count(_.persistent_id_y, sort = True)

(
    master_changes_raw
    >> group_by(_.persistent_id_y)
    >> mutate(n=_.shape[0])
    >> ungroup()
    >> arrange(-_.n)
    >> select(
        _.contains("artist"),
        _.contains("name"),
        _.contains("album"),
    )
)

,artist_x,artist_y,name_x,name_y,album_x,album_y
0,Frank Sinatra,Frank Sinatra,Got You Under My Skin,I've Got You Under My Skin,Songs For Swingin' Lovers,Ultimate Sinatra
1,Frank Sinatra,Frank Sinatra,I've Got You Under My Skin,I've Got You Under My Skin,Frank Sinatra Sings The Select Cole Porter [Bo...,Ultimate Sinatra
2,Frank Sinatra,Frank Sinatra,I've Got You Under My Skin,I've Got You Under My Skin,The Capitol Years [Disc 1],Ultimate Sinatra
3,Frank Sinatra,Frank Sinatra,I've Got You Under My Skin,I've Got You Under My Skin,Sinatra Reprise: The Very Good Years,Ultimate Sinatra
4,Frank Sinatra,Frank Sinatra,I've Got You Under My Skin,I've Got You Under My Skin,Gold!,Ultimate Sinatra
...,...,...,...,...,...,...
4736,Chicago,Chicago,Old Days,Old Days,The Heart Of Chicago 1967 - 1998 (Volume II),The Very Best of Chicago: Only the Beginning
4737,Young Jeezy,Young Jeezy,Too Many Commas ft. Birdman (Prod by Kenoe) (D...,Too Many Commas ft. Birdman (Prod by Kenoe) (D...,Its Tha World,Its Tha World
4738,Lil Wayne & Family,Lil Wayne & Family,Ace Hood Feat. Rick Ross - Realest Livin (Prod...,Ace Hood Feat. Rick Ross - Realest Livin (Prod...,All-Madden 22.0: Thanksgiving Edition (Leftovers),All-Madden 22.0: Thanksgiving Edition (Leftovers)
4739,David Essex,David Essex,Rock On,Rock On,Rock On,Rock On


## Update metadata

Metadata to change:
- date_added
- play_count
- skip_count
- genre

We will group by the new `persistent_id` and aggregate data for each of the above columns.
Then we will filter out any rows where there is no difference from the current data to create an output CSV that reflects the final changes.

#### date_added

In [16]:
date_added = (
    master_changes_raw
    >> group_by(_.persistent_id_y)
    >> summarize(
        date_added_x=_.date_added_x.astype("datetime64[s]").min(),
        date_added_y=_.date_added_y.astype("datetime64[s]").min(),
    )
    >> mutate(date_added=case_when({
        _.date_added_x < _.date_added_y: _.date_added_x,
        True: _.date_added_y
    }))
)

In [17]:
# Check for albums with that have different added_at dates - chances are most albums should be added at the same date.
(
    date_added
    >> inner_join(
        _,
        data2 >> transmute(_.album, _.artist, persistent_id_y=_.persistent_id),
        on="persistent_id_y",
    )
    >> group_by(_.artist, _.album)
    >> summarize(
        min_date_added=_.date_added.min(),
        max_date_added=_.date_added.max(),
    )
    >> ungroup()
    >> filter(_.min_date_added != _.max_date_added)
)

# I feel fine about this.

,artist,album,min_date_added,max_date_added
2,2 Chainz,B.O.A.T.S. II #METIME (Deluxe),2013-11-29 07:00:23,2014-02-12 19:26:58
3,2 Chainz,Based On a T.R.U. Story (Deluxe Version),2012-11-19 04:22:18,2015-08-07 01:31:39
5,2 Chainz,FreeBase,2014-05-08 00:30:00,2014-05-08 00:30:17
7,2Pac,Greatest Hits,2015-03-06 03:39:59,2015-03-06 03:40:26
11,50 Cent,Get Rich or Die Tryin',2011-08-17 17:03:47,2011-09-01 02:11:17
...,...,...,...,...
1655,Young Jeezy,Its Tha World,2012-12-15 15:09:25,2012-12-15 15:10:29
1657,Young Jeezy,TM:103 Hustlerz Ambition (Deluxe Version),2012-10-10 00:55:36,2012-11-16 23:26:11
1658,Young Jeezy,The Recession (Bonus Track Version),2011-08-17 17:04:17,2012-06-26 20:21:03
1669,Zapp & Roger,Zapp & Roger: All the Greatest Hits,2011-12-31 19:31:00,2014-06-25 02:11:51


#### Play Count / Skip Count

In [11]:
# If it is the same exact song, we want to keep the most recent number.
# If it is a different song (different filetype, different pid) sum them.

play_counts = (
    master_changes_raw
    >> mutate(
        play_count_y=_.play_count_y.fillna(0),
        play_count_x=_.play_count_x.fillna(0),
        skip_count_y=_.skip_count_y.fillna(0),
        skip_count_x=_.skip_count_x.fillna(0),
    )
    >> mutate(
        play_count=case_when(
            {
                _.src == "joined_on_pid": _.play_count_y,
                True: _.play_count_x + _.play_count_y,
            }
        ),
        skip_count=case_when(
            {
                _.src == "joined_on_pid": _.skip_count_y,
                True: _.skip_count_x + _.skip_count_y,
            }
        ),
    )
    >> group_by(_.persistent_id_y)
    >> summarize(play_count=_.play_count.sum(), skip_count=_.skip_count.sum())
    >> ungroup()
)

#### Genre

In [12]:
# It turns out, AM includes a bunch of garbage genres.
genre_counts = (
    data2
#     >> filter(_.kind == "Apple Music AAC audio file")
    >> count(_.genre, sort = True)
    >> arrange(_.genre)
)

In [13]:
from siuba.dply.forcats import fct_collapse

# Note: Manually reassign (in iTunes):
# Electronica
# Vocal
# Unknown/NA

genre_map = {
    "Alternative": "Europe",
    "Dance": [
        "Fitness & Workout",
        "House",
        "Trance",
    ],
    "Easy Listening": [
        "Vocal Jazz",
        "Vocal Pop",
    ],
    "Folk": "Traditional Folk",
    "Hip-Hop/Rap": [
        "East Coast Rap",
        "Gangsta Rap",
        "Hardcore Rap",
        "Hip Hop/Rap",
        "Hip-Hop/Rap",
        "Hip-Hop",
        "Hip-hop & Rap",
        "Rap",
    ],
    "Rock": [
        "Hard Rock",
        "Indie Rock",
        "Metal",
        "Pop/Rock",
        "Punk",
        "Soft Rock",
    ],
    "R&B/Soul": ["R&B", "Soul"],
}

In [14]:
genre_changes = (
    data2 
    >> filter(~_.genre.isna())
    >> mutate(genre=fct_collapse(_.genre, genre_map))
)

(
    genre_changes
    >> count(_.genre)
    >> arrange(_.genre)
)

,genre,n
0,Action & Adventure,2
1,Adult Contemporary,2
2,Alternative,320
3,Bluegrass,1
4,Blues,4
5,Children's Music,10
6,Christian & Gospel,3
7,Classical,44
8,Comedy,11
9,Country,159


#### Combine

In [23]:
master_changes = (
    genre_changes >> select(_.persistent_id, _.genre)
    >> full_join(_, play_counts >> rename(persistent_id="persistent_id_y"), on="persistent_id")
    >> full_join(_, date_added >> transmute(_.date_added, persistent_id=_.persistent_id_y), on="persistent_id")
    >> full_join(_, data2 >> select(_.artist, _.name, _.album, _.persistent_id), on="persistent_id")
)

master_changes = (
    master_changes
    >> mutate(
        date_added_secs=case_when({
            _.date_added.isna(): np.NaN,
            True: _.date_added.astype("int64") / 1000000000 # Convert from ns to s
        }),
#         date_added_secs=_.date_added.astype("int64") / 1000000000,
        play_count=_.play_count.astype("int64"),
        skip_count=_.skip_count.astype("int64"),
    )
)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [72]:
master_changes.to_csv("../data/master_changes.csv")
pickle.dump(master_changes, open("../data/master_changes.p", "wb"))

### Hunt for duplicates

In [ ]:
# TODO

## Why are some added dates missing from the 2019 lib?
Answer: Songs are included here even if `added_at` is NA. This happens when you add shared playlists without adding individual songs to your library.

In [5]:

# print(
#     data1.shape,
#     data2.shape,
#     joined.shape
#     )

print(
    data1.date_added.isna().sum(),
    data2.date_added.isna().sum()
    )

0 0


In [11]:

(
    data2
    >> group_by(_.artist)
    >> summarize(ttl_na=_.date_added.isna().sum())
    >> arrange(-_.ttl_na)
).head()

,artist,ttl_na
0,"""Weird Al"" Yankovic",0
1,10cc,0
2,2 Chainz,0
3,21 Savage,0
4,21 Savage & Metro Boomin,0


In [12]:
# data2 >> filter(_.artist == "Mura Masa")

Aha! Songs for shared playlists appear in the library but have not been "added". It is safe to exclude them.